In [1]:
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pendulum
from dateutil.relativedelta import relativedelta

from skill import config
from skill import plot

In [2]:
dir_dados = config.dir_dados
dir_img = config.dir_img

In [3]:
mes = ['', 'Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro','Dezembro']

In [4]:
rmse_bruto = xr.open_dataset(dir_dados.joinpath('cam3-rmse_prev_bruta','cam3_init1_rmse.nc')).__xarray_dataarray_variable__
rmse_ajustado = xr.open_dataset(dir_dados.joinpath('cam3-rmse_prev_ajustada','cam3_init1_rmse_ajustado.nc')).__xarray_dataarray_variable__
diff = rmse_ajustado - rmse_bruto

<GeoAxesSubplot:title={'center':'Diferença'}>

In [14]:
fig,ax = plt.subplots(1,3, figsize=(30,10), subplot_kw=dict(projection = ccrs.PlateCarree()))
ax[0].set_title('RMSE Bruta', fontsize=12)
ax[1].set_title('RMSE Ajustada', fontsize=12)
ax[2].set_title('Diferença', fontsize=12)
plot.plot_mesh(ax[0], rmse_bruto.sel(lead=1), vmin=0, vmax=300, cmap='hot_r')
plot.plot_mesh(ax[1], rmse_ajustado.sel(lead=1, month=2), vmin=0, vmax=300, cmap='hot_r')
plot.plot_mesh(ax[2], diff.sel(lead=1, month=2), vmin=-100, vmax=100, cmap='RdBu_r')


In [5]:
init = np.arange(1,13,1)
leads = np.arange(1,4,1)

dir_img.joinpath('cam3').mkdir(exist_ok=True)

for i in init:
    ds = xr.open_dataset(f'{dir_dados}/cam3-anom_corr-init{i}.nc')
    
    data = pendulum.now().set(year=1975, month=i, day=21)
    
    for lead in leads:
        
        lead_mes = (data + relativedelta(months=lead)).month

        fig,ax = plt.subplots(1,1, figsize=(10,10), subplot_kw=dict(projection = ccrs.PlateCarree()))
        
        plt.suptitle("Correlação CAM3.1 x GPCC \n2012-2020", x=0.45, y=0.9, fontsize=16)
        ax.set_title(f'CI e CC: {mes[i]}', loc='left')
        ax.set_title(f'Válido para {mes[lead_mes]}', loc='right')
        ax.text(-41.5,-31.7,'Lammoc - UFF', fontstyle='italic')

        plot.plot_mesh(ax, ds.__xarray_dataarray_variable__.sel(lead=lead))
        
        plt.close('all')
        
        fig.savefig(f'{dir_img}/cam3/cam3-corr_anom-init{i}_lead{lead}.png', bbox_inches='tight', facecolor='w', edgecolor='w')

KeyboardInterrupt: 